In [111]:
import pandas as pd

df = pd.read_excel('data_socio_economic_102_v20240607.xlsx') # https://tochno.st/datasets/regions_collection
print(df.head())

                       section  indicator_code             indicator_name  \
0  Торговля и услуги населению  Y4171501000000  Оборот розничной торговли   
1  Торговля и услуги населению  Y4171501000000  Оборот розничной торговли   
2  Торговля и услуги населению  Y4171501000000  Оборот розничной торговли   
3  Торговля и услуги населению  Y4171501000000  Оборот розничной торговли   
4  Торговля и услуги населению  Y4171501000000  Оборот розничной торговли   

  subsection           object_name object_level  object_oktmo  object_okato  \
0         CD  Российская Федерация       страна             0             0   
1         CD  Российская Федерация       страна             0             0   
2         CD  Российская Федерация       страна             0             0   
3         CD  Российская Федерация       страна             0             0   
4         CD  Российская Федерация       страна             0             0   

   year  indicator_value                                     i

In [112]:
df['object_level'].unique(), len(df)

(array(['страна', 'федеральный округ', 'регион'], dtype=object), 958911)

In [113]:
# Use only region rows
df_clean = df[df['object_level'] == 'регион']
len(df_clean)

868929

In [114]:
region_codes = {
    "Республика Адыгея": 1,
    "Республика Башкортостан": 2,
    "Республика Бурятия": 3,
    "Республика Алтай": 4,
    "Республика Дагестан": 5,
    "Республика Ингушетия": 6,
    "Кабардино-Балкарская Республика": 7,
    "Республика Калмыкия": 8,
    "Карачаево-Черкесская Республика": 9,
    "Республика Карелия": 10,
    "Республика Коми": 11,
    "Республика Марий Эл": 12,
    "Республика Мордовия": 13,
    "Республика Саха (Якутия)": 14,
    "Республика Северная Осетия — Алания": 15,
    "Республика Татарстан": 16,
    "Республика Тыва": 17,
    "Удмуртская Республика": 18,
    "Республика Хакасия": 19,
    "Чеченская Республика": 20,
    "Чувашская Республика": 21,
    "Алтайский край": 22,
    "Краснодарский край": 23,
    "Красноярский край": 24,
    "Приморский край": 25,
    "Ставропольский край": 26,
    "Хабаровский край": 27,
    "Амурская область": 28,
    "Архангельская область": 29,
    "Астраханская область": 30,
    "Белгородская область": 31,
    "Брянская область": 32,
    "Владимирская область": 33,
    "Волгоградская область": 34,
    "Вологодская область": 35,
    "Воронежская область": 36,
    "Ивановская область": 37,
    "Иркутская область": 38,
    "Калининградская область": 39,
    "Калужская область": 40,
    "Камчатский край": 41,
    "Кемеровская область": 42,
    "Кировская область": 43,
    "Костромская область": 44,
    "Курганская область": 45,
    "Курская область": 46,
    "Ленинградская область": 47,
    "Липецкая область": 48,
    "Магаданская область": 49,
    "Московская область": 50,
    "Мурманская область": 51,
    "Нижегородская область": 52,
    "Новгородская область": 53,
    "Новосибирская область": 54,
    "Омская область": 55,
    "Оренбургская область": 56,
    "Орловская область": 57,
    "Пензенская область": 58,
    "Пермский край": 59,
    "Псковская область": 60,
    "Ростовская область": 61,
    "Рязанская область": 62,
    "Самарская область": 63,
    "Саратовская область": 64,
    "Сахалинская область": 65,
    "Свердловская область": 66,
    "Смоленская область": 67,
    "Тамбовская область": 68,
    "Тверская область": 69,
    "Томская область": 70,
    "Тульская область": 71,
    "Тюменская область": 72,
    "Ульяновская область": 73,
    "Челябинская область": 74,
    "Забайкальский край": 75,
    "Ярославская область": 76,
    "Москва": 77,
    "Санкт-Петербург": 78,
    "Еврейская автономная область": 79,
    "Республика Крым": 82,
    "Ненецкий автономный округ": 83,
    "Ханты-Мансийский автономный округ — Югра": 86,
    "Чукотский автономный округ": 87,
    "Ямало-Ненецкий автономный округ": 89,
    "Севастополь": 92
}

In [115]:
# Use only stats for the whole region (drop "без автономных округов")
df_clean = df_clean[df_clean['object_name'].isin(region_codes.keys())]
df_clean['object_code'] = df_clean['object_name'].map(region_codes)
len(df_clean)

849301

In [116]:
# Use only rows with values
df_clean = df_clean[df_clean['indicator_value'] != -99999999]
df_clean = df_clean[df_clean['indicator_value'] != -77777777]
len(df_clean)

797813

In [117]:
# Use only recent stats
df_clean = df_clean[df_clean['year'] >= 2015]
len(df_clean)

303792

In [118]:
# Drop stats which are not complete year-wise
df_complete = (
    df_clean.groupby(['section', 'indicator_code', 'indicator_name', 'subsection',
       'object_name', 'object_level', 'object_oktmo', 'object_okato', 'comment', 'source'])
    .filter(lambda x: len(set(x['year'].unique())) >= 2022-2015+1)
)
len(df_complete)

267629

In [119]:
# Drop stats which are not complete region-wise
df_complete = (
    df_complete.groupby(['section', 'indicator_code', 'indicator_name', 'subsection',
       'object_level','comment', 'source'])
    .filter(lambda x: len(set(x['object_oktmo'].unique())) >= 80)
)
len(df_complete)

236090

In [120]:
df_complete.to_csv('df_complete.csv', index=False)

In [121]:
# Print all indicators for manual selection
for i in df_complete['indicator_name'].unique():
    print(i)

Оборот розничной торговли
Индексы физического объема оборота розничной торговли
Оборот розничной торговли на душу населения
Продажа товаров на розничных рынках и ярмарках
Оборот розничной торговли по торговым сетям
Структура оборота розничной торговли: Пищевые продукты, включая напитки, и табачные изделия
Структура оборота розничной торговли: Непродовольственные товары
Оборот общественного питания
Оборот общественного питания на душу населения
Оборот оптовой торговли: Всего
Оборот оптовой торговли: Оборот оптовой торговли организаций оптовой торговли
Объем платных услуг населению: Всего
Объем платных услуг населению: Индексы физического объема платных услуг населению
Объем платных услуг: На душу населения
Объем бытовых услуг населению: Всего
Объем бытовых услуг: На душу населения
Объем транспортных услуг населению: Всего
Объем транспортных услуг: На душу населения
Объем телекоммуникационных услуг населению: Всего
Объем телекоммуникационных услуг населению: На душу населения
Объем комму

In [122]:
allowed_indicators = [
    "Оборот розничной торговли на душу населения",
    "Объем платных услуг: На душу населения",
    "Среднедушевые денежные доходы населения",
    "Численность населения с денежными доходами ниже границы бедности/величины прожиточного минимума",
    "Потребительские расходы в среднем на душу населения",
    "Среднегодовая численность населения",
    "Соотношение мужчин и женщин",
    "Возрастной состав населения: Население в трудоспособном возрасте",
    "Коэффициенты естественного прироста населения на 1000 человек населения",
    "Коэффициенты миграционного прироста на 10 000 человек населения",
    "Число действующих кредитных организаций",
    "Число действующих филиалов кредитных организаций в субъекте",
    "Средства (вклады) юридических и физических лиц в рублях, привлеченные кредитными организациями: Всего",
    "Задолженность по кредитам в рублях, предоставленным кредитными организациями физическим лицам: Всего",
    "Посевные площади сельскохозяйственных культур",
    "Лесные ресурсы: Площадь, покрытая лесом",
    "Инвестиции в основной капитал на душу населения",
    "Стоимость основных фондов",
    "Ввод в действие основных фондов",
    "Степень износа основных фондов",
    "Использование цифровых технологий в организациях: Организации, использовавшие персональные компьютеры",
    "Использование цифровых технологий в организациях: Организации, использовавшие серверы",
    "Использование цифровых технологий в организациях: Организации, использовавшие локальные вычислительные сети",
    "Использование цифровых технологий в организациях: Организации, использовавшие «Облачные» сервисы",
    "Организации, имевшие веб-сайт",
    "Использование электронного документооборота в организациях: Организации, использовавшие электронный обмен данными между своими и внешними информационными системами, по форматам обмена",
    "Ввод в действие жилых домов: На 1000 человек населения",
    "Ввод в действие квартир: На 1000 человек населения",
    "Число организаций",
    "Оборот организаций",
    "Сальдированный финансовый результат (прибыль минус убыток) деятельности организаций",
    "Удельный вес убыточных организаций",
    "Кредиторская задолженность организаций: Просроченная",
    "Дебиторская задолженность организаций: Просроченная",
    "Среднегодовая численность занятых",
    "Уровень занятости населения в трудоспособном возрасте",
    "Уровень безработицы: Уровень безработицы в трудоспособном возрасте",
    "Потребность в работниках, заявленная работодателями в органы службы занятости населения"
]

In [123]:
# Remove extra indicators
df_allowed = df_complete[df_complete['indicator_name'].isin(allowed_indicators)]
len(df_allowed)

25348

In [124]:
# Pivot
df_pivoted = df_allowed.pivot_table(
    index=['object_code', 'year'],
    columns=['indicator_code'],
    values='indicator_value'
)

# Flatten
df_pivoted.columns = [f'{ind}' for ind in df_pivoted.columns]
df_pivoted = df_pivoted.reset_index()
df_pivoted.head()

,object_code,year,Y4170102000000,Y4170105000000,Y4170106020000,Y4170114000000,Y4170120000000,Y4170203000000,Y4170206000000,Y4170210020000,...,Y4171510030000,Y4171701010000,Y4171701020000,Y4171701030000,Y4171701050000,Y4171704000000,Y4171705020000,Y4171908000000,Y4171909010000,Y4171914010000
0,1,2015,462.6,1144.0,56.7,-0.5,121.0,155.1,66.474941,9.252594,...,25571.2,95.4,46.1,57.6,17.647059,50.2,51.8,5.0,15214.0,25545.0
1,1,2016,467.6,1142.0,56.2,-0.9,107.0,151.1,69.301128,9.277249,...,28026.6,96.3,48.7,59.9,20.300000,53.2,58.0,4.0,18087.0,23991.0
2,1,2017,471.2,1140.0,55.7,-2.0,77.0,152.1,67.912000,9.262121,...,29343.3,94.5,49.4,61.0,27.067669,52.1,63.0,3.0,20007.0,26147.0
3,1,2018,474.6,1140.0,55.2,-2.4,108.0,151.6,70.140914,8.942905,...,30159.7,92.0,50.5,62.4,27.620087,52.5,58.6,2.0,21338.0,29433.0
4,1,2019,482.1,1133.0,56.5,-3.2,258.0,152.0,70.284807,8.474892,...,31834.2,92.7,49.2,62.0,27.833895,57.0,58.7,2.0,23213.0,35988.0


In [125]:
df_pivoted.to_csv('df_pivoted.csv', index=False)

In [126]:
df_description = df_allowed[['indicator_code', 'section', 'indicator_name', 'indicator_unit', 'source']].drop_duplicates()
df_description.head()

,indicator_code,section,indicator_name,indicator_unit,source
4477,Y4171503000000,Торговля и услуги населению,Оборот розничной торговли на душу населения,"в фактически действовавших ценах, рублей",Регионы России. Социально-экономические показа...
27901,Y4171510030000,Торговля и услуги населению,Объем платных услуг: На душу населения,рублей,Регионы России. Социально-экономические показа...
54685,Y4170302000000,Уровень жизни населения,Среднедушевые денежные доходы населения,"в месяц, рублей",Регионы России. Социально-экономические показа...
99037,Y4170315000000,Уровень жизни населения,Численность населения с денежными доходами ниж...,в процентах от общей численности населения суб...,Регионы России. Социально-экономические показа...
101437,Y4170317000000,Уровень жизни населения,Потребительские расходы в среднем на душу насе...,"в месяц, рублей",Регионы России. Социально-экономические показа...


In [127]:
df_description.to_csv('df_description.csv', index=False)

In [128]:
!pip install clickhouse_driver

In [129]:
!wget "https://storage.yandexcloud.net/cloud-certs/RootCA.pem"

--2025-07-04 09:30:57--  https://storage.yandexcloud.net/cloud-certs/RootCA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1860 (1.8K) [application/x-x509-ca-cert]
Saving to: ‘RootCA.pem.1’

RootCA.pem.1        100%[===================>]   1.82K  --.-KB/s    in 0s      

2025-07-04 09:30:58 (369 MB/s) - ‘RootCA.pem.1’ saved [1860/1860]



In [ ]:
from clickhouse_driver import Client

# Create ClickHouse client
client = Client(
    host="r***-chp*****4g*gb***.***.yandexcloud.net",
    user="admin",
    password="admin",
    port=9440,
    secure=True,
    verify=True,
    ca_certs="RootCA.pem"
)

print(client.execute("SELECT version()"))

[('25.3.4.190',)]


In [131]:
dtype_mapping = {
    'int64': 'Int64',
    'float64': 'Float64',
    'object': 'String',
    'bool': 'UInt8',
    'datetime64[ns]': 'DateTime'
}

# Generate CREATE TABLE statement
def generate_create_table(df, table_name):
    columns = []
    for col, dtype in df.dtypes.items():
        ch_type = dtype_mapping.get(str(dtype), 'String')  # Default to String
        columns.append(f"{col} {ch_type}")

    create_sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n  " + ",\n  ".join(columns) + "\n) ENGINE = MergeTree() ORDER BY tuple()"
    return create_sql

In [132]:
print(generate_create_table(df_pivoted, 'db1.stat_features'))
print(generate_create_table(df_description, 'db1.stat_desc'))

CREATE TABLE IF NOT EXISTS db1.stat_features (
  object_code Int64,
  year Int64,
  Y4170102000000 Float64,
  Y4170105000000 Float64,
  Y4170106020000 Float64,
  Y4170114000000 Float64,
  Y4170120000000 Float64,
  Y4170203000000 Float64,
  Y4170206000000 Float64,
  Y4170210020000 Float64,
  Y4170211000000 Float64,
  Y4170302000000 Float64,
  Y4170315000000 Float64,
  Y4170317000000 Float64,
  Y4170901000000 Float64,
  Y4170902000000 Float64,
  Y4170903000000 Float64,
  Y4171002000000 Float64,
  Y4171101000000 Float64,
  Y4171102000000 Float64,
  Y4171103000000 Float64,
  Y4171104000000 Float64,
  Y4171105020000 Float64,
  Y4171106020000 Float64,
  Y4171304000000 Float64,
  Y4171337020000 Float64,
  Y4171403020000 Float64,
  Y4171404020000 Float64,
  Y4171503000000 Float64,
  Y4171510030000 Float64,
  Y4171701010000 Float64,
  Y4171701020000 Float64,
  Y4171701030000 Float64,
  Y4171701050000 Float64,
  Y4171704000000 Float64,
  Y4171705020000 Float64,
  Y4171908000000 Float64,
  Y41719

In [135]:
# Create table db1.stat_features
client.execute('''
CREATE TABLE IF NOT EXISTS db1.stat_features (
  object_code Int64,
  year Int64,
  Y4170102000000 Float64,
  Y4170105000000 Float64,
  Y4170106020000 Float64,
  Y4170114000000 Float64,
  Y4170120000000 Float64,
  Y4170203000000 Float64,
  Y4170206000000 Float64,
  Y4170210020000 Float64,
  Y4170211000000 Float64,
  Y4170302000000 Float64,
  Y4170315000000 Float64,
  Y4170317000000 Float64,
  Y4170901000000 Float64,
  Y4170902000000 Float64,
  Y4170903000000 Float64,
  Y4171002000000 Float64,
  Y4171101000000 Float64,
  Y4171102000000 Float64,
  Y4171103000000 Float64,
  Y4171104000000 Float64,
  Y4171105020000 Float64,
  Y4171106020000 Float64,
  Y4171304000000 Float64,
  Y4171337020000 Float64,
  Y4171403020000 Float64,
  Y4171404020000 Float64,
  Y4171503000000 Float64,
  Y4171510030000 Float64,
  Y4171701010000 Float64,
  Y4171701020000 Float64,
  Y4171701030000 Float64,
  Y4171701050000 Float64,
  Y4171704000000 Float64,
  Y4171705020000 Float64,
  Y4171908000000 Float64,
  Y4171909010000 Float64,
  Y4171914010000 Float64
) ENGINE = MergeTree() ORDER BY (object_code, year)
''')

[]

In [136]:
# Insert data
client.insert_dataframe(
    'INSERT INTO db1.stat_features VALUES',
    df_pivoted,
    settings={'async_insert': 1, 'wait_for_async_insert': 0, 'use_numpy': True}
)

765

In [137]:
# Fetch first few rows
result = client.execute(
    "SELECT * FROM db1.stat_features LIMIT 5",
    with_column_types=True
)

# Display results
headers = [col[0] for col in result[1]]
rows = result[0]
pd.DataFrame(rows, columns=headers).head()

,object_code,year,Y4170102000000,Y4170105000000,Y4170106020000,Y4170114000000,Y4170120000000,Y4170203000000,Y4170206000000,Y4170210020000,...,Y4171510030000,Y4171701010000,Y4171701020000,Y4171701030000,Y4171701050000,Y4171704000000,Y4171705020000,Y4171908000000,Y4171909010000,Y4171914010000
0,1,2015,462.6,1144.0,56.7,-0.5,121.0,155.1,66.474941,9.252594,...,25571.2,95.4,46.1,57.6,17.647059,50.2,51.8,5.0,15214.0,25545.0
1,1,2016,467.6,1142.0,56.2,-0.9,107.0,151.1,69.301128,9.277249,...,28026.6,96.3,48.7,59.9,20.300000,53.2,58.0,4.0,18087.0,23991.0
2,1,2017,471.2,1140.0,55.7,-2.0,77.0,152.1,67.912000,9.262121,...,29343.3,94.5,49.4,61.0,27.067669,52.1,63.0,3.0,20007.0,26147.0
3,1,2018,474.6,1140.0,55.2,-2.4,108.0,151.6,70.140914,8.942905,...,30159.7,92.0,50.5,62.4,27.620087,52.5,58.6,2.0,21338.0,29433.0
4,1,2019,482.1,1133.0,56.5,-3.2,258.0,152.0,70.284807,8.474892,...,31834.2,92.7,49.2,62.0,27.833895,57.0,58.7,2.0,23213.0,35988.0


In [ ]:
# Create table db1.stat_desc
client.execute('''
CREATE TABLE IF NOT EXISTS db1.stat_desc (
  indicator_code String,
  section String,
  indicator_name String,
  indicator_unit String,
  source String
) ENGINE = MergeTree() ORDER BY tuple()
''')

[]

In [ ]:
# Insert data
client.insert_dataframe(
    'INSERT INTO db1.stat_desc VALUES',
    df_description,
    settings={'async_insert': 1, 'wait_for_async_insert': 0, 'use_numpy': True}
)

37

In [ ]:
# Fetch first few rows
result = client.execute(
    "SELECT * FROM db1.stat_desc LIMIT 5",
    with_column_types=True
)

# Display results
headers = [col[0] for col in result[1]]
rows = result[0]
pd.DataFrame(rows, columns=headers).head()

,indicator_code,section,indicator_name,indicator_unit,source
0,Y4171503000000,Торговля и услуги населению,Оборот розничной торговли на душу населения,"в фактически действовавших ценах, рублей",Регионы России. Социально-экономические показа...
1,Y4171510030000,Торговля и услуги населению,Объем платных услуг: На душу населения,рублей,Регионы России. Социально-экономические показа...
2,Y4170302000000,Уровень жизни населения,Среднедушевые денежные доходы населения,"в месяц, рублей",Регионы России. Социально-экономические показа...
3,Y4170315000000,Уровень жизни населения,Численность населения с денежными доходами ниж...,в процентах от общей численности населения суб...,Регионы России. Социально-экономические показа...
4,Y4170317000000,Уровень жизни населения,Потребительские расходы в среднем на душу насе...,"в месяц, рублей",Регионы России. Социально-экономические показа...
